In [7]:
"""
The design of this comes from here:
http://outlace.com/Reinforcement-Learning-Part-3/
"""

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import RMSprop
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback

# Adding this per a suggestion by Tim Kelch.
# https://medium.com/@trkelch/this-post-is-great-possibly-the-best-tutorial-explanation-ive-found-thus-far-cf78886b5378#.w473ywtbw


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))


def neural_net(num_sensors, params, load=''):
    model = Sequential()

    # First layer.
    model.add(Dense(
        params[0], kernel_initializer='lecun_uniform', input_shape=(num_sensors,)
    ))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Second layer.
    model.add(Dense(params[1], kernel_initializer='lecun_uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Output layer.
    model.add(Dense(3, kernel_initializer='lecun_uniform'))
    model.add(Activation('linear'))

    rms = RMSprop()
    model.compile(loss='mse', optimizer=rms)

    if load:
        model.load_weights(load)

    return model


def lstm_net(num_sensors, load=False):
    model = Sequential()
    model.add(LSTM(
        output_dim=512, input_dim=num_sensors, return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(output_dim=512, input_dim=512, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=3, input_dim=512))
    model.add(Activation("linear"))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")

    return model

In [8]:
from carGame import carGame
import numpy as np
import random
import csv
from neuralNetwork import neural_net, LossHistory
import os.path
import timeit

NUM_INPUT = 2
GAMMA = 0.9  # Forgetting.
TUNING = False  # If False, just use arbitrary, pre-selected params.


def train_net(model, params):

    filename = params_to_filename(params)

    observe = 1000  # Number of frames to observe before training.
    epsilon = 1
    train_frames = 10000  # Number of frames to play.
    batchSize = params['batchSize']
    buffer = params['buffer']

    # Just stuff used below.
    max_car_distance = 0
    car_distance = 0
    t = 0
    data_collect = []
    replay = []  # stores tuples of (S, A, R, S').

    loss_log = []

    # Create a new game instance.
    game_state = carGame()

    # Get initial state by doing nothing and getting the state.
    _, state = game_state.update(0)

    # Let's time it.
    start_time = timeit.default_timer()

    # Run the frames.
    while t < train_frames:

        t += 1
        car_distance += 1

        # Choose an action.
        if random.random() < epsilon or t < observe:
            action = np.random.randint(0, 3)  # random
        else:
            # Get Q values for each action.
            qval = model.predict(state, batch_size=1)
            action = (np.argmax(qval))  # best

        # Take action, observe new state and get our treat.
        reward, new_state = game_state.update(action)

        # Experience replay storage.
        replay.append((state, action, reward, new_state))

        # If we're done observing, start training.
        if t > observe:

            # If we've stored enough in our buffer, pop the oldest.
            if len(replay) > buffer:
                replay.pop(0)

            # Randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)

            # Get training values.
            X_train, y_train = process_minibatch(minibatch, model)

            # Train the model on this batch.
            history = LossHistory()
            model.fit(
                X_train, y_train, batch_size=batchSize,
                nb_epoch=1, verbose=0, callbacks=[history]
            )
            loss_log.append(history.losses)

        # Update the starting state with S'.
        state = new_state

        # Decrement epsilon over time.
        if epsilon > 0.1 and t > observe:
            epsilon -= (1/train_frames)

        # We died, so update stuff.
        if reward == -500:
            print('crash reported');
            # Log the car's distance at this T.
            data_collect.append([t, car_distance])

            # Update max.
            if car_distance > max_car_distance:
                max_car_distance = car_distance

            # Time it.
            tot_time = timeit.default_timer() - start_time
            fps = car_distance / tot_time

            # Output some stuff so we can watch.
            print("Max: %d at %d\tepsilon %f\t(%d)\t%f fps" %
                  (max_car_distance, t, epsilon, car_distance, fps))

            # Reset.
            car_distance = 0
            start_time = timeit.default_timer()

        # Save the model every 25,000 frames.
        if t % 250 == 0:
            model.save_weights('saved-models/' + filename + '-' +
                               str(t) + '.h5',
                               overwrite=True)
            print("Saving model %s - %d" % (filename, t))

    # Log results after we're done all frames.
    log_results(filename, data_collect, loss_log)


def log_results(filename, data_collect, loss_log):
    # Save the results to a file so we can graph it later.
    with open('results/sonar-frames/learn_data-' + filename + '.csv', 'w') as data_dump:
        wr = csv.writer(data_dump)
        wr.writerows(data_collect)

    with open('results/sonar-frames/loss_data-' + filename + '.csv', 'w') as lf:
        wr = csv.writer(lf)
        for loss_item in loss_log:
            wr.writerow(loss_item)


def process_minibatch(minibatch, model):
    """This does the heavy lifting, aka, the training. It's super jacked."""
    X_train = []
    y_train = []
    # Loop through our batch and create arrays for X and y
    # so that we can fit our model at every step.
    for memory in minibatch:
        # Get stored values.
        old_state_m, action_m, reward_m, new_state_m = memory
        # Get prediction on old state.
        old_qval = model.predict(old_state_m, batch_size=1)
        # Get prediction on new state.
        newQ = model.predict(new_state_m, batch_size=1)
        # Get our best move. I think?
        maxQ = np.max(newQ)
        y = np.zeros((1, 3))
        y[:] = old_qval[:]
        # Check for terminal state.
        if reward_m != -500:  # non-terminal state
            update = (reward_m + (GAMMA * maxQ))
        else:  # terminal state
            update = reward_m
        # Update the value for the action we took.
        y[0][action_m] = update
        X_train.append(old_state_m.reshape(NUM_INPUT,))
        y_train.append(y.reshape(3,))

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def params_to_filename(params):
    return str(params['nn'][0]) + '-' + str(params['nn'][1]) + '-' + \
            str(params['batchSize']) + '-' + str(params['buffer'])


def launch_learn(params):
    filename = params_to_filename(params)
    print("Trying %s" % filename)
    # Make sure we haven't run this one.
    if not os.path.isfile('results/sonar-frames/loss_data-' + filename + '.csv'):
        # Create file so we don't double test when we run multiple
        # instances of the script at the same time.
        open('results/sonar-frames/loss_data-' + filename + '.csv', 'a').close()
        print("Starting test.")
        # Train.
        model = neural_net(NUM_INPUT, params['nn'])
        train_net(model, params)
    else:
        print("Already tested.")


if __name__ == "__main__":
    if TUNING:
        param_list = []
        nn_params = [[164, 150], [256, 256],
                     [512, 512], [1000, 1000]]
        batchSizes = [40, 100, 400]
        buffers = [10000, 50000]

        for nn_param in nn_params:
            for batchSize in batchSizes:
                for buffer in buffers:
                    params = {
                        "batchSize": batchSize,
                        "buffer": buffer,
                        "nn": nn_param
                    }
                    param_list.append(params)

        for param_set in param_list:
            launch_learn(param_set)

    else:
        nn_param = [164, 150]
        params = {
            "batchSize": 100,
            "buffer": 50000,
            "nn": nn_param
        }
        model = neural_net(NUM_INPUT, nn_param)
        train_net(model, params)

/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
4.846030199428071
0.0
4.123050877794753
4.123049905790481
4.12304890766876
0.0
4.12304788248548
4.12304682925344
0.0
4.369782074602256
0.0
4.367740807644509
4.365638611754321
4.363473068832036
4.36124164124675
4.358941664778058
4.356570341075724
4.354124729597288
4.351601738985266
0.0
0.0
4.573052623934586
0.0
4.5629283687287625
4.552456908327542
4.54162303610525
0.0
0.0
0.0
3.751867005602037
3.723649972101043
0.0
0.0
0.0
2.8922032679808467
2.838096829624533
2.7827736768465456
2.7262218528303492
2.668430796668474
2.6093914708295074
2.549096491413536
2.4875402604873216
2.424719099691629
2.360631384209796
2.295277676082719
2.228660855755379
0.0
1.1611083675535951
0.0
1.0914680327958308
1.0211817210090999
0.9502837779631932
0.0
-0.12070040901670609
0.0
-0.19161566991715517
-0.2624153255171109
-500
crash reported
Max: 57 at 57	epsilon 1.000000	(57)	19.317710 fps
-500
crash reported
Max: 57 at 58	epsilon 1.000000	(1)	19.540443 fps
-500
crash reported
Max: 5

-500
crash reported
Max: 79 at 240	epsilon 1.000000	(1)	18.794312 fps
-500
crash reported
Max: 79 at 241	epsilon 1.000000	(1)	19.189263 fps
-500
crash reported
Max: 79 at 242	epsilon 1.000000	(1)	20.362587 fps
-500
crash reported
Max: 79 at 243	epsilon 1.000000	(1)	21.979794 fps
-500
crash reported
Max: 79 at 244	epsilon 1.000000	(1)	21.155230 fps
-500
crash reported
Max: 79 at 245	epsilon 1.000000	(1)	21.499414 fps
-500
crash reported
Max: 79 at 246	epsilon 1.000000	(1)	21.287210 fps
-500
crash reported
Max: 79 at 247	epsilon 1.000000	(1)	20.113994 fps
-500
crash reported
Max: 79 at 248	epsilon 1.000000	(1)	21.623122 fps
-500
crash reported
Max: 79 at 249	epsilon 1.000000	(1)	21.433141 fps
-500
crash reported
Max: 79 at 250	epsilon 1.000000	(1)	20.530367 fps
Saving model 164-150-100-50000 - 250
-500
crash reported
Max: 79 at 251	epsilon 1.000000	(1)	17.692197 fps
-500
crash reported
Max: 79 at 252	epsilon 1.000000	(1)	20.763591 fps
-500
crash reported
Max: 79 at 253	epsilon 1.000000	(

-500
crash reported
Max: 137 at 463	epsilon 1.000000	(1)	21.271745 fps
-500
crash reported
Max: 137 at 464	epsilon 1.000000	(1)	18.943607 fps
-500
crash reported
Max: 137 at 465	epsilon 1.000000	(1)	20.521637 fps
-500
crash reported
Max: 137 at 466	epsilon 1.000000	(1)	19.961777 fps
-500
crash reported
Max: 137 at 467	epsilon 1.000000	(1)	18.481886 fps
-500
crash reported
Max: 137 at 468	epsilon 1.000000	(1)	18.985434 fps
-500
crash reported
Max: 137 at 469	epsilon 1.000000	(1)	19.186040 fps
-500
crash reported
Max: 137 at 470	epsilon 1.000000	(1)	20.328452 fps
-500
crash reported
Max: 137 at 471	epsilon 1.000000	(1)	20.318012 fps
-500
crash reported
Max: 137 at 472	epsilon 1.000000	(1)	19.817876 fps
-500
crash reported
Max: 137 at 473	epsilon 1.000000	(1)	20.754250 fps
-500
crash reported
Max: 137 at 474	epsilon 1.000000	(1)	20.221542 fps
-500
crash reported
Max: 137 at 475	epsilon 1.000000	(1)	21.307142 fps
-500
crash reported
Max: 137 at 476	epsilon 1.000000	(1)	20.069750 fps
-500
c

-500
crash reported
Max: 165 at 708	epsilon 1.000000	(1)	19.724751 fps
-500
crash reported
Max: 165 at 709	epsilon 1.000000	(1)	20.577702 fps
-500
crash reported
Max: 165 at 710	epsilon 1.000000	(1)	20.754980 fps
-500
crash reported
Max: 165 at 711	epsilon 1.000000	(1)	22.202408 fps
-500
crash reported
Max: 165 at 712	epsilon 1.000000	(1)	19.966733 fps
-500
crash reported
Max: 165 at 713	epsilon 1.000000	(1)	21.028935 fps
-500
crash reported
Max: 165 at 714	epsilon 1.000000	(1)	18.169487 fps
-500
crash reported
Max: 165 at 715	epsilon 1.000000	(1)	20.793595 fps
-500
crash reported
Max: 165 at 716	epsilon 1.000000	(1)	19.351950 fps
-500
crash reported
Max: 165 at 717	epsilon 1.000000	(1)	20.846386 fps
-500
crash reported
Max: 165 at 718	epsilon 1.000000	(1)	18.696342 fps
-500
crash reported
Max: 165 at 719	epsilon 1.000000	(1)	22.230023 fps
-500
crash reported
Max: 165 at 720	epsilon 1.000000	(1)	20.562434 fps
0.6478696437101235
0.6046312878315234
0.5612195317748387
0.0
0.0
-0.420979871

...

-500
crash reported
Max: 188 at 9375	epsilon 0.162500	(1)	9.243462 fps
-500
crash reported
Max: 188 at 9376	epsilon 0.162400	(1)	9.168991 fps
-500
crash reported
Max: 188 at 9377	epsilon 0.162300	(1)	8.724379 fps
-500
crash reported
Max: 188 at 9378	epsilon 0.162200	(1)	8.676881 fps
-500
crash reported
Max: 188 at 9379	epsilon 0.162100	(1)	8.967793 fps
-500
crash reported
Max: 188 at 9380	epsilon 0.162000	(1)	9.221744 fps
-500
crash reported
Max: 188 at 9381	epsilon 0.161900	(1)	9.137492 fps
-500
crash reported
Max: 188 at 9382	epsilon 0.161800	(1)	9.518819 fps
-500
crash reported
Max: 188 at 9383	epsilon 0.161700	(1)	10.044711 fps
-500
crash reported
Max: 188 at 9384	epsilon 0.161600	(1)	10.536815 fps
-500
crash reported
Max: 188 at 9385	epsilon 0.161500	(1)	10.316039 fps
-500
crash reported
Max: 188 at 9386	epsilon 0.161400	(1)	10.419131 fps
-500
crash reported
Max: 188 at 9387	epsilon 0.161300	(1)	9.574571 fps
-500
crash reported
Max: 188 at 9388	epsilon 0.161200	(1)	9.202987 fps
-5

crash reported
Max: 188 at 9552	epsilon 0.144800	(1)	9.374130 fps
-500
crash reported
Max: 188 at 9553	epsilon 0.144700	(1)	9.699235 fps
-500
crash reported
Max: 188 at 9554	epsilon 0.144600	(1)	9.347434 fps
-500
crash reported
Max: 188 at 9555	epsilon 0.144500	(1)	9.597950 fps
-500
crash reported
Max: 188 at 9556	epsilon 0.144400	(1)	9.398600 fps
-500
crash reported
Max: 188 at 9557	epsilon 0.144300	(1)	9.408430 fps
-500
crash reported
Max: 188 at 9558	epsilon 0.144200	(1)	9.478834 fps
-500
crash reported
Max: 188 at 9559	epsilon 0.144100	(1)	9.591292 fps
-500
crash reported
Max: 188 at 9560	epsilon 0.144000	(1)	9.355641 fps
-500
crash reported
Max: 188 at 9561	epsilon 0.143900	(1)	9.156652 fps
-500
crash reported
Max: 188 at 9562	epsilon 0.143800	(1)	9.631837 fps
-500
crash reported
Max: 188 at 9563	epsilon 0.143700	(1)	9.409032 fps
-500
crash reported
Max: 188 at 9564	epsilon 0.143600	(1)	9.525132 fps
-500
crash reported
Max: 188 at 9565	epsilon 0.143500	(1)	9.257322 fps
-500
crash 

-500
crash reported
Max: 188 at 9717	epsilon 0.128300	(1)	9.010664 fps
3.9033346253507943
0.0
0.0
0.0
-500
crash reported
Max: 188 at 9722	epsilon 0.127800	(5)	9.251129 fps
-500
crash reported
Max: 188 at 9723	epsilon 0.127700	(1)	9.063750 fps
-500
crash reported
Max: 188 at 9724	epsilon 0.127600	(1)	9.060270 fps
-500
crash reported
Max: 188 at 9725	epsilon 0.127500	(1)	9.502687 fps
-500
crash reported
Max: 188 at 9726	epsilon 0.127400	(1)	8.604688 fps
-500
crash reported
Max: 188 at 9727	epsilon 0.127300	(1)	8.900429 fps
-500
crash reported
Max: 188 at 9728	epsilon 0.127200	(1)	9.117128 fps
-500
crash reported
Max: 188 at 9729	epsilon 0.127100	(1)	9.212098 fps
-500
crash reported
Max: 188 at 9730	epsilon 0.127000	(1)	9.244787 fps
-500
crash reported
Max: 188 at 9731	epsilon 0.126900	(1)	8.999603 fps
-500
crash reported
Max: 188 at 9732	epsilon 0.126800	(1)	9.433961 fps
-500
crash reported
Max: 188 at 9733	epsilon 0.126700	(1)	9.236058 fps
-500
crash reported
Max: 188 at 9734	epsilon 0

-500
crash reported
Max: 188 at 9978	epsilon 0.102200	(1)	9.218130 fps
-500
crash reported
Max: 188 at 9979	epsilon 0.102100	(1)	9.144975 fps
-500
crash reported
Max: 188 at 9980	epsilon 0.102000	(1)	9.235684 fps
-500
crash reported
Max: 188 at 9981	epsilon 0.101900	(1)	9.278940 fps
-500
crash reported
Max: 188 at 9982	epsilon 0.101800	(1)	9.321324 fps
0.8791998828010037
0.8432985196272966
0.8072273112201174
0.0
0.770992517534296
0.7346005568008422
0.0
0.0
-0.2540388162119598
0.0
-1.2356590893881503
-1.2626614858099856
-1.2894735136377449
0.0
0.0
0.0
0.0
0.0
Saving model 164-150-100-50000 - 10000


FileNotFoundError: [Errno 2] No such file or directory: 'results/sonar-frames/learn_data-164-150-100-50000.csv'

In [ ]:
"""
Once a model is learned, use this to play it.
"""

from carGame import carGame
import numpy as np
from neuralNetwork import neural_net

NUM_SENSORS = 2


def play(model):

    car_distance = 0
    game_state = carGame()

    # Do nothing to get initial.
    _, state = game_state.update(0)

    # Move.
    while True:
        car_distance += 1

        # Choose action.
        action = (np.argmax(model.predict(state, batch_size=1)))

        # Take action.
        _, state = game_state.update(action)

        # Tell us something.
        if car_distance % 1000 == 0:
            print("Current distance: %d frames." % car_distance)


if __name__ == "__main__":
    saved_model = 'saved-models/164-150-100-50000-10000.h5'
    model = neural_net(NUM_SENSORS, [164, 150], saved_model)
    play(model)

/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
4.846030199428071
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
